## Q1 - Extract name, age, gender, fiscal region

In [ ]:
SELECT
    client_data->'Result'->0->'BasicData'->>'Name'               AS name,
    (client_data->'Result'->0->'BasicData'->>'Age')::INT         AS age,
    client_data->'Result'->0->'BasicData'->>'Gender'             AS gender,
    client_data->'Result'->0->'BasicData'->>'TaxIdFiscalRegion'  AS fiscal_region
FROM kyc_data;


## Q2 - Clients with IMPOSTO A PAGAR in 2024

In [ ]:
SELECT
    client_data->'Result'->0->'BasicData'->>'Name'         AS name,
    client_data->'Result'->0->'BasicData'->>'TaxIdNumber'  AS tax_id,
    client_data->'Result'->0->'FinancialData'->'IncomeEstimates'->>'BIGDATA_V2' AS income_estimate
FROM kyc_data,
     jsonb_array_elements(
        client_data->'Result'->0->'FinancialData'->'TaxReturns'
     ) AS tax_return
WHERE tax_return->>'Year' = '2024'
  AND tax_return->>'Status' ILIKE '%IMPOSTO A PAGAR%';


## Q3 - Relationships + Has lawsuits

In [ ]:
SELECT
    client_data->'Result'->0->'BasicData'->>'Name' AS name,

    COALESCE(
        (client_data->'Result'->0->'RelatedPeople'->>'TotalSpouses')::INT, 0
    ) AS total_spouses,

    COALESCE(
        (client_data->'Result'->0->'RelatedPeople'->>'TotalRelatives')::INT, 0
    ) AS total_relatives,

    CASE
        WHEN COALESCE(
            (client_data->'Result'->0->'Processes'->>'TotalLawsuits')::INT, 0
        ) > 0
        THEN 'YES'
        ELSE 'NO'
    END AS has_lawsuits
FROM kyc_data;


## Q4 - Sanctions + Classification

In [ ]:
SELECT
    client_data->'Result'->0->'BasicData'->>'Name'        AS name,
    client_data->'Result'->0->'BasicData'->>'TaxIdNumber' AS tax_id,

    COALESCE(
        (client_data->'Result'->0->'KycData'->>'Last90DaysSanctions')::INT, 0
    ) AS sanctions_90d,

    COALESCE(
        (client_data->'Result'->0->'KycData'->>'Last180DaysSanctions')::INT, 0
    ) AS sanctions_180d,

    CASE
        WHEN client_data->'Result'->0->'KycData'->>'IsCurrentlySanctioned' = 'true'
        THEN 'YES'
        ELSE 'NO'
    END AS currently_sanctioned,

    CASE
        WHEN COALESCE(
            (client_data->'Result'->0->'KycData'->>'Last180DaysSanctions')::INT, 0
        ) > 0
        THEN 'ATTENTION'
        ELSE 'OK'
    END AS classification
FROM kyc_data;


## Q5 - Family profile + Electoral donor

In [ ]:
SELECT  
    client_data->'Result'->0->'BasicData'->>'Name' AS name,
    (client_data->'Result'->0->'BasicData'->>'Age')::INT AS age,

    client_data->'Result'->0->'BasicData'
        ->'MaritalStatusData'->>'MaritalStatus' AS marital_status,

    COALESCE(
        (client_data->'Result'->0->'RelatedPeople'->>'TotalSpouses')::INT, 0
    ) AS total_spouses,

    COALESCE(
        (client_data->'Result'->0->'RelatedPeople'->>'TotalRelatives')::INT, 0
    ) AS total_relatives,

    CASE
        WHEN COALESCE(
            (client_data->'Result'->0->'RelatedPeople'->>'TotalRelatives')::INT, 0
        ) >= 5
        THEN 'LARGE_FAMILY'
        WHEN COALESCE(
            (client_data->'Result'->0->'RelatedPeople'->>'TotalSpouses')::INT, 0
        ) >= 1
        THEN 'MARRIED'
        ELSE 'SINGLE'
    END AS family_situation,

    CASE
        WHEN client_data->'Result'->0->'KycData'->>'IsCurrentlyElectoralDonor' = 'true'
        THEN 'DONOR'
        ELSE 'NON_DONOR'
    END AS electoral_donor_status
FROM kyc_data;
